In [ ]:
# Core
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
# from matplotlib.pyplot.bar_label
import seaborn as sns;
from scipy import stats
import glob
import random
import datetime
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import matplotlib.cm as cm
import os
import pickle
# from  datasist.structdata import detect_outliers
from tqdm import tqdm
# Core
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set(rc={'figure.figsize':[7,7]},font_scale=1.2)
from datetime import date,timedelta
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# Pre Processing
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# Regressors
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
# Error Metrics 
from sklearn.metrics import r2_score #r2 square
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay ,classification_report
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score


#classefication
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier #stacstic gradient descent clasifeier
import graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#crossvalidation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import ConfusionMatrixDisplay
#clustring 
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
#hyper parameter tunning
from sklearn.model_selection import GridSearchCV
#pca
from sklearn.decomposition import PCA
#clustring
from sklearn.cluster import KMeans
from warnings import filterwarnings
filterwarnings("ignore")

# 1. Data Processing

In [ ]:
seed = 42
np.random.seed =seed

In [ ]:
df = pd.read_csv("../input/buyer-correlation-matrix/suggestion_correlation_matrix.csv")
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
cols =df.columns
sns.heatmap(df[cols].isnull(), cmap='viridis')

# Onehot encoding for categorical columns


In [ ]:
#print unique value of each categorical data
print(df['main_category'].unique()) 
print(df['main_relationship'].unique()) 
print(df['location'].unique()) 


In [ ]:
#count value
df['main_category'].value_counts() 
df['main_category'].replace('', 'Other', inplace=True)

In [ ]:
df['main_relationship'].value_counts() 

In [ ]:
df['main_relationship'].replace('', 'Other', inplace=True)

In [ ]:
df['location'].value_counts() 

In [ ]:
df['location'].replace('', 'Other', inplace=True)

In [ ]:
#onehot
# importing libraries 
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder 


# Converting type of columns to category 
df['main_category'] = df['main_category'].astype('category') 
df['main_relationship'] = df['main_relationship'].astype('category') 
df['location'] = df['location'].astype('category') 

# Assigning numerical values and storing it in another columns 
df['main_category_enc'] = df['main_category'].cat.codes 
df['main_relationship_enc'] = df['main_relationship'].cat.codes 
df['location_enc'] = df['location'].cat.codes 

# Create an instance of One-hot-encoder 
enc = OneHotEncoder() 

# Passing encoded columns 

enc_data = pd.DataFrame(enc.fit_transform(df[['main_relationship_enc', 'main_category_enc','location_enc']]).toarray()) 

# Merge with main 
new_df = df.join(enc_data) 

print(new_df.info()) 


In [ ]:
enc_df= new_df.copy()
enc_df = enc_df[['spent_month','location_enc','main_category_enc','main_relationship_enc','Recency_month','frequency_month','total_spending','total_order','basket_size','avg_fsv','avg_prv']]

In [ ]:
enc_df.corr()

In [ ]:
sns.heatmap(data=enc_df.corr(),annot = True)

# 2. Kmeans Clustering

# Standardization

In [ ]:
cluster_data= new_df.copy()
cluster_data = cluster_data[['customer_id','spent_month','location_enc','main_category_enc','main_relationship_enc','Recency_month','frequency_month','total_spending','total_order','basket_size','avg_fsv','avg_prv']]

In [ ]:
std_df = cluster_data
std_df.drop(['customer_id'],axis=1, inplace= True)  #drop id column

scaled_features = StandardScaler().fit_transform(std_df)
scaled_features_df = pd.DataFrame(scaled_features, columns=std_df.columns)
scaled_features_df.head()

# Dimensionality reduction

In [ ]:
pca = PCA(n_components=4)
pca.fit(scaled_features_df)
pca_df = pd.DataFrame(pca.transform(scaled_features_df), columns=(["column1", "column2", "column3", "column4"]))

# Clustering and Elbow method

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
# elbow method to find numbers of clusters to make.
kmeans = KMeans()
elbow_method = KElbowVisualizer((kmeans), k=10)
elbow_method.fit(pca_df)
elbow_method.show();

# Kmeans Method

In [ ]:
ac = AgglomerativeClustering(n_clusters=4)

ac_clstering = ac.fit_predict(pca_df)
pca_df["cluster"] = ac_clstering

df["cluster"]= ac_clstering # adding cluster feature to the orignal dataframe.
df['cluster'] = 'cluster'+ ' ' + df['cluster'].astype('str')

df.to_csv('sencond_clustering.csv',encoding='utf-8-sig')
df.head()

In [ ]:
# !pip install summarytools

In [ ]:
from summarytools import dfSummary
dfSummary(df[['spent_month','Recency_month','frequency_month','main_category','main_relationship','main_working_hour','location','total_spending','total_order','basket_size','cluster','avg_fsv','avg_prv']])

Test

In [ ]:
import pandas as pd
df1 = pd.read_csv("../input/list-order-id/Final Round - Working File - list_orderid.csv")
df1

In [ ]:
import pandas as pd
df2 = pd.read_csv("../input/occassion-define-2/Final Round - Working File - Sheet23 (1).csv")
df2

In [ ]:
# Convert date strings to datetime objects
df1['receive_day'] = pd.to_datetime(df1['receive_day'])
df2['start_day'] = pd.to_datetime(df2['start_day'])
df2['end_day'] = pd.to_datetime(df2['end_day'])

In [ ]:

# Merge the DataFrames on a common column
merged_df = pd.merge(df1, df2, left_on='receive_day', right_on='start_day', how='left')
merged_df['start_day'].unique()

In [ ]:
def check_conditions(row):
    if row['receive_day'] >= row['start_day'] and row['receive_day'] <= row['end_day']:
        return row['occassion_name']
    else:
        return None 

In [ ]:
# Apply the custom function to create the 'occassion_name' column
merged_df['occassion_name'] = merged_df.apply(check_conditions, axis=1)

# Drop unnecessary columns
result_df = merged_df[['order_id','receive_day', 'occassion_name']]

# If you want to fill NaN values in the 'occassion_name' column with a default value, you can use fillna
result_df['occassion_name'].fillna("Not in range", inplace=True)

# Display the result
print(result_df)

In [ ]:
result_df.to_csv('final_occasion.csv',encoding='utf-8-sig')